https://www.philipzucker.com/compose_datalog/

Bdddbbdd
Or using SDD?

datakanren should use tries


CLP using z3 guards
Bottom up CLP https://en.wikipedia.org/wiki/Constraint_logic_programming#Bottom-up_evaluation
A(0).
A(X):-X>0.
A(X):-X=Y+1, A(Y).
fomrulog
Generation loops



revive snakelog for benchmarks


Negation - Fancy forall negation. Where did we talk about that? arntzenius?
Aggregates

LAttice / semiring
https://duckdb.org/2025/05/23/using-key.html
https://sqlite.org/lang_conflict.html

provenance?

Egglog / SLOG via 
`==` is at the same level as <= ?

extension of env

inject comments into.
Tuple together DatalogQuery

class DatalogQuery():
    q : str
    def 

env.pretty
edge(x,y) & edge(y,z)


class Env[A]:
    # Gamma |- A
    value : A 
    cols
    code


greatest fixpoint for deleting rules. DELETE FROM insterad of INSERT INTO. The delta is reversed?
Still insert into new


Monotonic graph rewriting.


Postgres has more powerful recusrive ctes? https://www.postgresql.org/docs/current/queries-with.html#QUERIES-WITH-RECURSIVE
https://news.ycombinator.com/item?id=45035618  https://vadimtropashko.wordpress.com/%e2%80%9csql-design-patterns%e2%80%9d-book/about/ some interesting tree and graph encodings. path strings

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=test user=postgres")
conn.close()

OperationalError: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?


You know, honestly the pyuthon impl is more flexible for prototyping. I'm not going to be making big stuff.

In [ ]:
from typing import Callable
type Var = str
type Env = dict[Var, object]
type DB = dict[str, set[tuple(object,...)]]
type Query = Callable[[DB], list[Env]]
from dataclasses import dataclass

@dataclass(frozen=True)
class Var():
    name : str

def reldecl(name: str) -> Callable[..., Query]:
    def rel(*args) -> Query:
        def res(db: DB) -> list[Env]:
            R = db[name]
            envs = []
            for row in R:
                env = {}
                for arg, val in zip(args, row):
                    if isinstance(arg, Var):
                        if arg in env and env[arg] != val:
                            break
                        env[arg] = val
                    else:
                        if arg != val:
                            break
                else:
                    envs.append(env)
            return envs
        return res
    return rel

def conj(*queries: Query) -> Query:
    def res(db : DB) -> list[Env]:
        envs = [{}]
        for query in queries:
            R = query(db)
            newenvs = []        
            for env1 in R:
                for env2 in envs:
                    if all(env1[k] == env2[k] for k in set(env1.keys()) & set(env2.keys())):
                        newenvs.append(env1 | env2)
            envs = newenvs
        return envs
    return res

type Action = Callable[[DB, Query], DB]

def headdecl(name : str):
    def rel(*args) -> Action:
        def res(query : Query, db : DB) -> DB:
            envs = query(db)
            Rold = db.get(name, set())
            Rnew = {tuple(env[arg] if isinstance(arg,Var) else arg for arg in args) for env in envs}
            return {**db, **{name : Rnew | Rold}}
        return res
    return rel